# Installing Net2Brain

In [ ]:
!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

# Step 3: Evaluation

In this tutorial notebook, we'll showcase how to leverage the evaluation capabilities of Net2Brain and visualize the resulting data. You can choose from three evaluation metrics:

1. "RSA"
2. "Weighted RSA"
3. "Searchlight"

Each module generates a pandas DataFrame, which can be seamlessly integrated with the toolbox's built-in plotting functionality.

### Representational Similiratiy Analysis (RSA) Evaluation

In [ ]:
from net2brain.evaluations.rsa import RSA
from net2brain.utils.download_datasets import load_dataset

First load the ROIs

In [ ]:
stimuli_path, roi_path = load_dataset("bonner_pnas2017")

In [ ]:
model_rdms = "AlexNet_RDM" 
brain_rdms = roi_path

# Start RSA
evaluation_alexnet = RSA(model_rdms, brain_rdms, model_name="AlexNet")

# Evaluation - Returns a pandas dataframe
dataframe1 = evaluation_alexnet.evaluate() 

# Show results
display(dataframe1)


## Visualizing RSA Evaluation Results

The integrated plotting functionality of the toolbox allows you to easily visualize evaluation results. To achieve this, initialize the class with a list of DataFrames obtained from the evaluation. Make sure that each DataFrame:

1. Contains the same ROIs, signifying that each test was performed on the same brain RDMs.
2. Has a distinct model name, which can be set manually or through the "model_name" parameter during evaluation (as mentioned earlier).

Here's an example of how to plot the data using a single DataFrame:

In [ ]:
from net2brain.evaluations.plotting import Plotting

plotter = Plotting([dataframe1])
results_dataframe = plotter.plot()

You can choose between `metric="R2"` (default) or `metric="R"`

In [ ]:
results_dataframe = plotter.plot(metric="R")

You can also choose between plotting the best layers or `plot_all_layers()`.

In [ ]:
results_dataframe = plotter.plot_all_layers(metric="R2")

## Visualizing RSA Evaluation Results - Multiple models

As previously mentioned, you can also plot multiple models in a single plot. To do this, simply include additional DataFrames in the list:

        Plotting([dataframe1, dataframe2, dataframe3])



In [ ]:
# Start RSA for AlexNet
evaluation_alexnet = RSA("AlexNet_RDM", brain_rdms, save_path="./", model_name="AlexNet")
dataframe2 = evaluation_alexnet.evaluate() 

# Start RSA for ResNet50
evaluation_resnet = RSA("ResNet50_RDM", brain_rdms, save_path="./", model_name="ResNet50")
dataframe1 = evaluation_resnet.evaluate() 


plotter = Plotting([dataframe1,dataframe2])
results_dataframe = plotter.plot()

## Visualizing RSA Evaluation Results - Multiple models with significance


Furthermore, you might be interested in determining whether one model is significantly better than another, and not merely due to random variation. In this case, you can utilize the `compare_model` functionality provided by the toolbox. Use the following syntax:

        ttest, sig_pairs = eval_1.compare_model(eval_2)

If you wish to display the significance as well, use the parameter pairs=[].

In [ ]:
# Comparing statistical significance
ttest, sig_pairs = evaluation_alexnet.compare_model(evaluation_resnet)
print(sig_pairs)

# Plotting with significance
plotter = Plotting([dataframe1,dataframe2])
results_dataframe = plotter.plot(pairs=sig_pairs)

### WRSA Evaluation
In addition to the standard RSA, Net2Brain also supports weighted RSA (WRSA) as an evaluation metric. WRSA allows for the incorporation of weights into the analysis, providing an alternative approach to evaluating model performance and examining the relationship between neural representations and computational models.

In [ ]:
from net2brain.evaluations.weighted_rsa import WRSA

# Start RSA
evaluation = WRSA(model_rdms, brain_rdms, save_path="./", model_name="ResNet50")

# Evaluation - Returns a pandas dataframe
dataframe1 = evaluation.evaluate() 

display(dataframe1)

### Searchlight
The toolbox offers the capability to perform searchlight analysis using Searchlight RDMs in the [ROI, subject, stimuli, stimuli] format. Please note that this toolbox does not include RDMs for testing purposes. However, if you have access to RDMs, you are welcome to use this functionality to conduct searchlight analysis.

In [ ]:
from net2brain.evaluations.searchlight import Searchlight
model_rdms = "-"
searchlight_rdm = "-"

evaluation = Searchlight(model_rdms, searchlight_rdm, save_path="./")
evaluation.evaluate()